# Baseado nos testes anteriores com esta base e com a base ChestXRay:

* Extrair features para numpy com imageaugmented bem "suave" (teste 01b3)
   * Rodar com maxpool e com avgpool para poder comparar
* Rodar keras_tuner e comparar resultados com melhor resultado da rede simples
* Criar matriz e testar busca interativamente com as features da imagenet
* Reduzir com PCA ou T-SNE e testar se prejudica busca
* Testar clusterização (para colocar busca em produção mesmo com dezenas de milhões de imagens)

### Os resultados da rede simples treinada do zero foram bons, mas caso os resultados com rede pré treinada na imagenet sejam bons, pode ser um método universal para vários classificadores, buscas e análises pré extrair features via rede pré treinada, salvando em Banco de Dados.

In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import RMSprop

from utils import report_classes
import time


caminho_bases = os.path.join('..', 'bases', 'vazios')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

MODEL_DIR = '../models_featureextraction'
LOG_DIR = '../logs/02c2/'
SIZE = (224, 224)
destino_bases = os.path.join('..', 'bases', 'vazios_featureextraction')
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
if not os.path.exists(caminho_bases):
    os.mkdir(caminho_bases)


# Callbacks

In [2]:
tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02c2.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2,
                              verbose=1, mode='min')


# Model for feature extraction

In [7]:
# create the base pre-trained model
base_model = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='max')

# DataGenerators

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.05,
    brightness_range=[0.95, 1.05],
    fill_mode='nearest',
    horizontal_flip=True,
)
train_generator = train_datagen.flow_from_directory(
    caminho_train,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

Found 20845 images belonging to 2 classes.


In [9]:
validation_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.05,
    brightness_range=[0.95, 1.05],
    fill_mode='nearest',
    horizontal_flip=True,
)

validation_generator = validation_datagen.flow_from_directory(
    caminho_test,
    target_size=SIZE,
    batch_size=64,
    class_mode='binary',
    shuffle=False
)

Found 2317 images belonging to 2 classes.


# Extract features to np

In [3]:
FEATURES_DIR = destino_bases

In [4]:
# Se tiver sido gerado anteriormente, basta ler do disco
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'rb') as features_in:
    features_train = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'rb') as y_in:
    y_train = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'rb') as features_in:
    features_test = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'rb') as y_in:
    y_test = np.load(y_in)

In [35]:
def extract_features(generator, model):
    generator.reset()
    n_images = len(generator.filenames)
    batch_size = generator.batch_size
    m = batch_size *  (n_images // batch_size)  # Arredondar para não ficar espaço vazio
    n = model.output.shape[1]
    features = np.zeros((m, n), np.float32)
    y_ = np.zeros((m, 1), np.float32)
    print(n_images, m, batch_size, m // batch_size)
    for ind in range(m // batch_size):
        batch, y_batch = next(generator)
        features_batch = base_model.predict(batch)
        features[ind * batch_size: (ind * batch_size) + batch_size, :] = features_batch
        y_[ind * batch_size: (ind * batch_size) + batch_size, 0] = y_batch
    print('last batch setted elements %s:%s' % ((ind * batch_size), ((ind * batch_size) + batch_size)))
    return features, y_

In [37]:
features_train, y_train = extract_features(train_generator, base_model)

20845 20800 64 325
last batch setted elements 20736:20800


In [38]:
print(features_train.shape)
print(y_train.shape)

(20800, 1024)
(20800, 1)


In [36]:
features_test, y_test = extract_features(validation_generator, base_model)

2317 2304 64 36
last batch setted elements 2240:2304


In [39]:
print(features_test.shape)
print(y_test.shape)

(2304, 1024)
(2304, 1)


In [40]:
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'bw') as features_out:
    np.save(features_out, features_train)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'bw') as y_out:
    np.save(y_out, y_train)

In [41]:
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'bw') as features_out:
    np.save(features_out, features_test)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'bw') as y_out:
    np.save(y_out, y_test)

# Model for feature extraction (AvgPooling)

In [42]:
# create the base pre-trained model
base_model_avg = DenseNet121(weights='imagenet',
                         input_shape=(*SIZE, 3), 
                         include_top=False,
                         pooling='avg')

# Extract features to np (AvgPooling)

In [10]:
# Se tiver sido gerado anteriormente, basta ler do disco
with open(os.path.join(FEATURES_DIR, 'features_train_avg.np'), 'rb') as features_in:
    features_train_avg = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train_avg.np'), 'rb') as y_in:
    y_train_avg = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test_avg.np'), 'rb') as features_in:
    features_test_avg = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test_avg.np'), 'rb') as y_in:
    y_test_avg = np.load(y_in)

In [43]:
features_train_avg, y_train_avg = extract_features(train_generator, base_model_avg)

20845 20800 64 325
last batch setted elements 20736:20800


In [44]:
print(features_train_avg.shape)
print(y_train_avg.shape)

(20800, 1024)
(20800, 1)


In [45]:
features_test_avg, y_test_avg = extract_features(validation_generator, base_model_avg)

2317 2304 64 36
last batch setted elements 2240:2304


In [46]:
print(features_test_avg.shape)
print(y_test_avg.shape)

(2304, 1024)
(2304, 1)


In [47]:
with open(os.path.join(FEATURES_DIR, 'features_train_avg.np'), 'bw') as features_out:
    np.save(features_out, features_train_avg)
with open(os.path.join(FEATURES_DIR, 'y_train_avg.np'), 'bw') as y_out:
    np.save(y_out, y_train_avg)

In [48]:
with open(os.path.join(FEATURES_DIR, 'features_test_avg.np'), 'bw') as features_out:
    np.save(features_out, features_test_avg)
with open(os.path.join(FEATURES_DIR, 'y_test_avg.np'), 'bw') as y_out:
    np.save(y_out, y_test_avg)

# Liberar memória

In [49]:
del base_model
del base_model_avg

# Model RandomSearch KerasTuner

In [86]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=128,
                           activation='relu',
                           input_shape=(1024,)))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(units=128,
                           activation='relu'))
    model.add(tf.keras.layers.Dropout(hp.Choice('lrate',
                                      values=[0.2, 0.4])))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[0.0001, 0.00004, 0.00001])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [79]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=4,
    executions_per_trial=2,
    directory='kerastunerlogs',
    project_name='Vazios')

In [80]:
tuner.search_space_summary()

# Tuner - Teste com MaxPool

In [81]:
tuner.search(x=features_train,
             y=y_train,
             epochs=10,
             validation_data=(features_test, y_test),
            )

W0710 12:10:39.228868 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.157958). Check your callbacks.
W0710 12:10:39.291945 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.104327). Check your callbacks.


Name,Best model,Current model
accuracy,0.9501,0.9501
val_loss,0.1445,0.1445
loss,0.1635,0.1635
val_accuracy,0.954,0.954


W0710 12:11:46.608629 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.
W0710 12:11:46.638159 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.
W0710 12:11:46.678763 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.
W0710 12:11:46.700974 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.106839). Check your callbacks.


Name,Best model,Current model
accuracy,0.9549,0.9549
val_loss,0.1402,0.1402
loss,0.1461,0.1461
val_accuracy,0.9557,0.9557


Oracle triggered exit


In [82]:
tuner.results_summary()

# Tuner - Teste com AvgPool

In [83]:
tuner.search(x=features_train_avg,
             y=y_train_avg,
             epochs=10,
             validation_data=(features_test_avg, y_test_avg),
            )

W0710 12:13:16.168517 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.200158). Check your callbacks.
W0710 12:13:16.202365 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.109022). Check your callbacks.


W0710 12:13:35.043132 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.109354). Check your callbacks.
W0710 12:13:35.259618 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112996). Check your callbacks.
W0710 12:13:35.408587 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112996). Check your callbacks.
W0710 12:13:35.523991 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112996). Check your callbacks.
W0710 12:13:35.559537 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107065). Check your callbacks.
W0710 12:13:35.689697 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107065). Check your callbacks.
W0710 12:13:35.719903 139904768079616 callbacks.py:241] Method (on_tra

W0710 12:13:49.451974 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.119044). Check your callbacks.
W0710 12:13:49.608206 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.138525). Check your callbacks.
W0710 12:13:49.729732 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.138525). Check your callbacks.
W0710 12:13:49.766310 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.138525). Check your callbacks.
W0710 12:13:49.778784 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.127418). Check your callbacks.
W0710 12:13:49.791177 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.107937). Check your callbacks.


W0710 12:14:32.717909 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112083). Check your callbacks.
W0710 12:14:32.837350 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.112083). Check your callbacks.
W0710 12:14:32.963488 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114096). Check your callbacks.
W0710 12:14:33.090873 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.114168). Check your callbacks.
W0710 12:14:33.219648 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.115007). Check your callbacks.
W0710 12:14:33.356442 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.115007). Check your callbacks.
W0710 12:14:33.384125 139904768079616 callbacks.py:241] Method (on_tra

W0710 12:14:55.397021 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.303431). Check your callbacks.
W0710 12:14:55.409013 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.155027). Check your callbacks.
W0710 12:15:03.769032 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.797059 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.810311 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.820936 139904768079616 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.135326). Check your callbacks.
W0710 12:15:03.832063 139904768079616 callbacks.py:241] Method (on_tra

Name,Best model,Current model
accuracy,0.9549,0.933
val_loss,0.1402,0.1718
loss,0.1461,0.2085
val_accuracy,0.9557,0.9481


Oracle triggered exit


In [84]:
tuner.results_summary()

# Final Training and validation

In [6]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(units=128,
                       activation='relu',
                       input_shape=(1024,)))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
    loss='binary_crossentropy',
    metrics=['accuracy'])


In [16]:
epoch = 43

In [61]:
val_loss = 0.13
model.load_weights(
    os.path.join(MODEL_DIR,
                 'Transfermodelweights02c2.{:02d}-{:.2f}.hdf5'.format(epoch, val_loss)
                ))

In [9]:
history = model.fit(x=features_train,
                    y=y_train,
                    batch_size=128,
                    initial_epoch=epoch,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test, y_test),
                    callbacks=[early_stop, mcp_save, reduce_lr, tensorboard_logs],
                   )

W0711 09:23:02.496668 140111535253248 deprecation.py:323] From /home/ivan/pybr/projeto/venv/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 20800 samples, validate on 2304 samples
Epoch 1/200


W0711 09:23:04.328726 140111535253248 callbacks.py:241] Method (on_train_batch_end) is slow compared to the batch update (0.108159). Check your callbacks.


20800/20800 - 2s - loss: 0.4658 - accuracy: 0.7889 - val_loss: 0.2374 - val_accuracy: 0.9214
Epoch 2/200
20800/20800 - 0s - loss: 0.2719 - accuracy: 0.9017 - val_loss: 0.2006 - val_accuracy: 0.9319
Epoch 3/200
20800/20800 - 0s - loss: 0.2317 - accuracy: 0.9227 - val_loss: 0.1855 - val_accuracy: 0.9397
Epoch 4/200
20800/20800 - 0s - loss: 0.2097 - accuracy: 0.9319 - val_loss: 0.1757 - val_accuracy: 0.9440
Epoch 5/200
20800/20800 - 0s - loss: 0.1992 - accuracy: 0.9362 - val_loss: 0.1679 - val_accuracy: 0.9462
Epoch 6/200
20800/20800 - 0s - loss: 0.1889 - accuracy: 0.9401 - val_loss: 0.1643 - val_accuracy: 0.9462
Epoch 7/200
20800/20800 - 0s - loss: 0.1834 - accuracy: 0.9407 - val_loss: 0.1587 - val_accuracy: 0.9470
Epoch 8/200
20800/20800 - 0s - loss: 0.1783 - accuracy: 0.9437 - val_loss: 0.1557 - val_accuracy: 0.9510
Epoch 9/200
20800/20800 - 0s - loss: 0.1712 - accuracy: 0.9473 - val_loss: 0.1535 - val_accuracy: 0.9510
Epoch 10/200
20800/20800 - 0s - loss: 0.1684 - accuracy: 0.9477 - v

In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00004),
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [14]:
mcp_save_avg = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02c2avg.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')

history = model.fit(x=features_train_avg,
                    y=y_train_avg,
                    batch_size=128,
                    initial_epoch=epoch,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test_avg, y_test_avg),
                    callbacks=[early_stop, mcp_save_avg, reduce_lr, tensorboard_logs],
                   )

Train on 20800 samples, validate on 2304 samples
Epoch 1/200
20800/20800 - 1s - loss: 0.1282 - accuracy: 0.9594 - val_loss: 0.1297 - val_accuracy: 0.9583
Epoch 2/200
20800/20800 - 0s - loss: 0.1266 - accuracy: 0.9588 - val_loss: 0.1291 - val_accuracy: 0.9583
Epoch 3/200
20800/20800 - 0s - loss: 0.1250 - accuracy: 0.9588 - val_loss: 0.1293 - val_accuracy: 0.9592
Epoch 4/200

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
20800/20800 - 0s - loss: 0.1258 - accuracy: 0.9591 - val_loss: 0.1295 - val_accuracy: 0.9579
Epoch 5/200
20800/20800 - 0s - loss: 0.1231 - accuracy: 0.9601 - val_loss: 0.1279 - val_accuracy: 0.9588
Epoch 6/200
20800/20800 - 0s - loss: 0.1213 - accuracy: 0.9600 - val_loss: 0.1273 - val_accuracy: 0.9588
Epoch 7/200
20800/20800 - 0s - loss: 0.1217 - accuracy: 0.9596 - val_loss: 0.1276 - val_accuracy: 0.9583
Epoch 8/200

Epoch 00008: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
20800/20800 - 0s - loss: 0.1212 - accurac

In [44]:
y_pred = model.predict(features_test)

y_pred = y_pred > 0.5

classes = {0: 'nvazio', 1: 'vazio'}
report_classes(classes, y_test, y_pred)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 1077 	 89
vazio 	 11 	 1127
              precision    recall  f1-score   support

         0.0       0.99      0.92      0.96      1166
         1.0       0.93      0.99      0.96      1138

    accuracy                           0.96      2304
   macro avg       0.96      0.96      0.96      2304
weighted avg       0.96      0.96      0.96      2304



In [49]:
y_pred_train = model.predict(features_train)

y_pred_train = y_pred_train > 0.5

report_classes(classes, y_train, y_pred_train)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 9745 	 749
vazio 	 40 	 10266
              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96     10494
         1.0       0.93      1.00      0.96     10306

    accuracy                           0.96     20800
   macro avg       0.96      0.96      0.96     20800
weighted avg       0.96      0.96      0.96     20800



In [62]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
mcp_save_weights = ModelCheckpoint(
    os.path.join(MODEL_DIR, 'Transfermodelweights02c2-classweights.{epoch:02d}-{val_loss:.2f}.hdf5'),
                 save_best_only=True, monitor='val_loss', mode='min'
)
class_weight = {0: 3., 1: 1.}
history = model.fit(x=features_train,
                    y=y_train,
                    batch_size=128,
                    class_weight=class_weight,
                    initial_epoch=0,
                    epochs=200,
                    verbose=2,
                    validation_data=(features_test, y_test),
                    callbacks=[early_stop, mcp_save_weights, reduce_lr, tensorboard_logs],
                   )

Train on 20800 samples, validate on 2304 samples
Epoch 1/200
20800/20800 - 1s - loss: 0.2577 - accuracy: 0.9542 - val_loss: 0.1488 - val_accuracy: 0.9540
Epoch 2/200
20800/20800 - 0s - loss: 0.2534 - accuracy: 0.9544 - val_loss: 0.1513 - val_accuracy: 0.9544
Epoch 3/200

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
20800/20800 - 0s - loss: 0.2475 - accuracy: 0.9556 - val_loss: 0.1516 - val_accuracy: 0.9540
Epoch 4/200
20800/20800 - 0s - loss: 0.2496 - accuracy: 0.9544 - val_loss: 0.1532 - val_accuracy: 0.9536
Epoch 5/200
20800/20800 - 1s - loss: 0.2474 - accuracy: 0.9543 - val_loss: 0.1484 - val_accuracy: 0.9531
Epoch 6/200
20800/20800 - 1s - loss: 0.2490 - accuracy: 0.9550 - val_loss: 0.1498 - val_accuracy: 0.9540
Epoch 7/200

Epoch 00007: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
20800/20800 - 1s - loss: 0.2470 - accuracy: 0.9551 - val_loss: 0.1521 - val_accuracy: 0.9527
Epoch 8/200
20800/20800 - 1s - loss: 0.2475 - accuracy

In [63]:
y_pred = model.predict(features_test)

y_pred = y_pred > 0.5

classes = {0: 'nvazio', 1: 'vazio'}
report_classes(classes, y_test, y_pred)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 1095 	 71
vazio 	 36 	 1102
              precision    recall  f1-score   support

         0.0       0.97      0.94      0.95      1166
         1.0       0.94      0.97      0.95      1138

    accuracy                           0.95      2304
   macro avg       0.95      0.95      0.95      2304
weighted avg       0.95      0.95      0.95      2304



In [64]:
y_pred_train = model.predict(features_train)

y_pred_train = y_pred_train > 0.5

report_classes(classes, y_train, y_pred_train)

{0: 'nvazio', 1: 'vazio'}
R/P 	 nvazio 	 vazio 
nvazio 	 9918 	 576
vazio 	 217 	 10089
              precision    recall  f1-score   support

         0.0       0.98      0.95      0.96     10494
         1.0       0.95      0.98      0.96     10306

    accuracy                           0.96     20800
   macro avg       0.96      0.96      0.96     20800
weighted avg       0.96      0.96      0.96     20800

